In [5]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [9]:
train_df = pd.read_csv("./data/jigsaw/train.csv")
test_df = pd.read_csv("./data/jigsaw/test.csv")
test_labels_df = pd.read_csv("./data/jigsaw/test_labels.csv")
identities = pd.read_csv("./data/adjectives_people.txt")

In [3]:
test_df = pd.merge(test_df, test_labels_df)

In [4]:
train_df_short = train_df.drop(train_df.columns[3:8], axis=1)
train_df_short

,id,comment_text,toxic
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0


In [5]:
test_df_short = test_df.drop(test_df.columns[3:8], axis=1)

In [10]:
identity_array = identities.to_numpy()
identity_array = np.append(identity_array, 'lesbian')
identity_list = identity_array.tolist()
identity_list

['gay',
 'bisexual',
 'transgender',
 'trans',
 'queer',
 'lgbt',
 'lgbtq',
 'homosexual',
 'straight',
 'heterosexual',
 'male',
 'female',
 'nonbinary',
 'african',
 'african american',
 'black',
 'white',
 'european',
 'hispanic',
 'latino',
 'latina',
 'latinx',
 'mexican',
 'canadian',
 'american',
 'asian',
 'indian',
 'middle eastern',
 'chinese',
 'japanese',
 'christian',
 'muslim',
 'jewish',
 'buddhist',
 'catholic',
 'protestant',
 'sikh',
 'taoist',
 'old',
 'older',
 'young',
 'younger',
 'teenage',
 'millenial',
 'middle aged',
 'elderly',
 'blind',
 'deaf',
 'paralyzed',
 'lesbian']

In [7]:
from tqdm import tqdm
for row_index, row in tqdm(enumerate(train_df_short.itertuples())):
    for index, identity in enumerate(identity_list):
        if identity in row[2]:
            train_df_short.at[row_index, identity] = 1
        else:
            train_df_short.at[row_index, identity] = 0
#print(train_df_short)

159571it [00:49, 3202.63it/s]


In [8]:
# test df add 1s and 0s
for row_index, row in tqdm(enumerate(test_df_short.itertuples())):
    for index, identity in enumerate(identity_list):
        if identity in row[2]:
            test_df_short.at[row_index, identity] = 1
        else:
            test_df_short.at[row_index, identity] = 0

153164it [00:49, 3104.32it/s]


In [9]:
test_df_short = test_df_short[test_df_short["toxic"] != -1]
test_df_short

,id,comment_text,toxic,gay,bisexual,transgender,trans,queer,lgbt,lgbtq,...,young,younger,teenage,millenial,middle aged,elderly,blind,deaf,paralyzed,lesbian
5,0001ea8717f6de06,Thank you for understanding. I think very high...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,000247e83dcc1211,:Dear god this site is horrible.,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153150,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
153151,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
153154,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
153155,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [178]:
from pathlib import Path  
test_df_short.to_csv(Path("./data/test_df.csv"))
train_df_short.to_csv(Path("./data/train_df.csv"))

In [11]:
test_df_short = pd.read_csv("./data/test_df_short.csv")
train_df_short = pd.read_csv("./data/train_df.csv")

In [11]:
# for training data set:
# id - sentence - toxic - 50 cols with 0 if identity is not there, 1 if identity is there

# Blindness

In [12]:
# Adding identity column to train_df_short (either works I think)
for row_index, row in tqdm(enumerate(train_df_short.itertuples())):
    train_df_short.at[row_index, "identity"] = 0
    for i in range(4,54):
        if row[i] == 1:
            train_df_short.at[row_index, "identity"] = 1

159571it [00:02, 60980.53it/s]


In [13]:
# Adding identity column to train_df_short (either works I think)
for row_index in tqdm(range(len(train_df_short))):
    train_df_short.at[row_index, "identity"] = max([train_df_short.at[row_index, identity] for identity in identity_list])

100%|██████████| 159571/159571 [00:24<00:00, 6500.72it/s]


In [14]:
# Creating train_df_blind by dropping all identiy columns from train_df_short
train_df_blind = train_df_short.drop(columns = identity_list)

In [15]:
# Replacing identities in comment text with an identity token
token = "identity"
for row_index in tqdm(range(len(train_df_blind))):
    if train_df_short.at[row_index, "identity"] == 1:
        for identity in identity_list:
            if train_df_short.at[row_index, identity] == 1 :
                train_df_blind.at[row_index, "comment_text"] = train_df_blind.at[row_index, "comment_text"].replace(identity, token)

100%|██████████| 159571/159571 [00:03<00:00, 47562.33it/s]


In [18]:
train_df_blind.to_csv(Path("./data/train_df_blind.csv"))

# Augmentation

In [16]:
train_df_synthetic = pd.DataFrame().reindex_like(train_df_short)
synth_row_index = 0
for row_index in tqdm(range(len(train_df_blind))):
    if train_df_blind.at[row_index, "identity"] == 1:
        for identity in identity_list:
            if train_df_short.at[row_index, identity] == 1:
                for diff_identity in identity_list:
                    if diff_identity != identity:
                        train_df_synthetic.loc[synth_row_index] = train_df_short.loc[row_index]
                        train_df_synthetic.at[synth_row_index, "comment_text"] = train_df_short.at[row_index, "comment_text"].replace(identity, diff_identity)
                        synth_row_index += 1
            

  4%|▍         | 6434/159571 [13:32<5:22:27,  7.92it/s] 


KeyboardInterrupt: 

In [33]:
train_df_synthetic = pd.DataFrame().reindex_like(train_df_short)
synth_row_index = 0
for row_index in tqdm(range(42,44)):
    for identity in identity_list:
        print(train_df_short.where(train_df_short[identity] == 0, comment_text)

 50%|█████     | 1/2 [00:01<00:01,  1.90s/it]

       Unnamed: 0                id  \
0               0  0000997932d777bf   
1               1  000103f0d9cfb60f   
2               2  000113f07ec002fd   
3               3  0001b41b1c6bb37e   
4               4  0001d958c54c6e35   
...           ...               ...   
159566     159566  ffe987279560d7ff   
159567     159567  ffea4adeee384e90   
159568     159568  ffee36eab5c267c9   
159569     159569  fff125370e4aaaf3   
159570     159570  fff46fc426af1f9a   

                                             comment_text toxic  gay bisexual  \
0       Explanation\nWhy the edits made under my usern...     0  0.0      0.0   
1       D'aww! He matches this background colour I'm s...     0  0.0      0.0   
2       Hey man, I'm really not trying to edit war. It...     0  0.0      0.0   
3       "\nMore\nI can't make any real suggestions on ...     0  0.0      0.0   
4       You, sir, are my hero. Any chance you remember...     0  0.0      0.0   
...                                          

100%|██████████| 2/2 [00:02<00:00,  1.42s/it]

       Unnamed: 0                id  \
0               0  0000997932d777bf   
1               1  000103f0d9cfb60f   
2               2  000113f07ec002fd   
3               3  0001b41b1c6bb37e   
4               4  0001d958c54c6e35   
...           ...               ...   
159566     159566  ffe987279560d7ff   
159567     159567  ffea4adeee384e90   
159568     159568  ffee36eab5c267c9   
159569     159569  fff125370e4aaaf3   
159570     159570  fff46fc426af1f9a   

                                             comment_text toxic  gay bisexual  \
0       Explanation\nWhy the edits made under my usern...     0  0.0      0.0   
1       D'aww! He matches this background colour I'm s...     0  0.0      0.0   
2       Hey man, I'm really not trying to edit war. It...     0  0.0      0.0   
3       "\nMore\nI can't make any real suggestions on ...     0  0.0      0.0   
4       You, sir, are my hero. Any chance you remember...     0  0.0      0.0   
...                                          

In [ ]:
from pathlib import Path  
train_df_synthetic.to_csv(Path("./data/train_df_synthetic.csv"))